# Hour Media total
Пример отчета по расчету аудитории по всему интернету с почасовой разбивкой.

# Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к API Cross Web и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.crossweb import tasks as cwt
from mediascope_api.crossweb import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с API Cross Web
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.CrossWebTask()
cats = cwc.CrossWebCats()

In [ ]:
# вывод возможных статистик для отчета
# cats.get_hour_media_total_unit().get('statistics')

In [ ]:
# вывод возможных срезов для отчета
# cats.get_hour_media_total_unit().get('slices')

In [ ]:
# вывод возможных фильтров для отчета
# cats.get_hour_media_total_unit().get('filters')

## Задание

### Расчет аудитории по всему интернету

In [ ]:
# Задаем название для отображения в DataFrame
project_name = 'Total'

# Задаем период
# Период указывается в виде списка ('Начало', 'Конец'). Можно указать несколько периодов
date_filter = [('2025-01-01', '2025-01-15')]

# Задаем фильтр по типам пользования интернетом
usetype_filter = [1,2,3]

# Задаем фильтр по географии, в нашем случае он не требуется
geo_filter = None

# Задаем фильтр по демографии
demo_filter = None
#demo_filter = 'ageGroup IN (1,2,3)'

# Задаем почасовой фильтр. Можно указать как группы часов (например, ночные или дневные часы), так и конкретный час:
# hour_filter = 'eventHour > 5'
# Ночные часы: 
#hour_filter = 'eventHour >= 1 AND eventHour <= 6'
# Конкретный час: 
# hour_filter = 'eventHour = 13'
# Если необходимо расчитать весь день:
hour_filter = 'eventHour >= 1 AND eventHour <= 23'

# Указываем список срезов, чтобы сформировать структуру расчета
slices = ["researchMonth", "eventHour"]

# Указываем список статистик для расчета
statistics = ['reach', 'reachPer']

scales = None


# Формируем задание для API Cross Web в формате JSON
task_json = mtask.build_task('hour-total', task_name=project_name, date_filter=date_filter,                                
                             geo_filter=geo_filter, demo_filter=demo_filter, 
                             hour_filter=hour_filter,
                             slices=slices, statistics=statistics,
                             scales=scales)

# Отправляем задание на расчет и ждем выполнения
task = mtask.wait_task(mtask.send_task(task_json))

# Получаем результат
df_result = mtask.result2table(mtask.get_result(task), project_name = project_name)
df_result

In [ ]:
# для отмены задания (если требуется)
# mtask.cancel_tasks(['33ef65e2-12e8-4b82-851b-4a40b7ad506c'])

In [ ]:
# вывод текста задания (если требуется)
# task_json

## Сохраняем в Excel

In [ ]:
df_info = mtask.get_report_info()

with pd.ExcelWriter(mtask.get_excel_filename('hour-audience-total')) as writer:
    df_result.to_excel(writer, sheet_name='Report', index=False)
    df_info.to_excel(writer, sheet_name='Info', index=False)